In [12]:
!pip install sqlalchemy

     ---------------------------------------- 2.0/2.0 MB 2.1 MB/s eta 0:00:00
     ------------------------------------- 192.2/192.2 kB 11.4 MB/s eta 0:00:00


In [1]:
# Importar bibiotecas
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
###
from src import config
import time
from datetime import datetime
from tqdm import tqdm
from src import dw_utils as dw

In [14]:
### atualzar lista de links no sitemap
BASE_URL = 'https://www.gruposhopmix.com/'
## obter lista de urls a partir do index

content = requests.get(config.URL_INDEX).text

### processar o html obtido 
### extracts all loc tags from content
soup = bs(content, 'html.parser')
all_locs = soup.find_all('loc')
urls = pd.DataFrame(columns = ["links_catalogo"], data=all_locs)

### remove keywords from urls
keywords = ['pagina','sao-paulo', 'todos-os-produtos', 'os-mais-vendidos', 'novidades']
urls = urls[~urls["links_catalogo"].str.contains('|'.join(keywords))]
## drop lines that is equal to the base url
urls = urls.loc[urls["links_catalogo"] != BASE_URL]
### obter dataset final com urls
urls.reset_index(inplace=True, drop=True)

c:\Users\bcesa\anaconda3\envs\market_scrapers\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [18]:

def extract_data_from_html(soup):
    
    
    nome = soup.find('meta', property='og:title')['content']
    codigo = soup.find('meta', attrs={'name': 'twitter:data1'})['content']
    try:
        preco = soup.find('strong', attrs = 'preco-promocional cor-principal titulo')['data-sell-price']
    except TypeError:
        preco = "N/A"
    disponibilidade = soup.find('meta', attrs={'name': 'twitter:data2'})['content']
    descricao = soup.find('meta', attrs={'name': 'description'})['content']
    try:
        disponibilidade_entrega = soup.find('span', class_='disponibilidade disp-entrega').find('b').text
    except AttributeError:
        disponibilidade_entrega = 'Não disponível'
    try:
        estoque = soup.find('b', class_='qtde_estoque').text
    except AttributeError:
        estoque = 'Não disponível'
    imagens = [img['src'] for img in soup.find_all('img') if 'produto' in img['src']]
    
    return {
        'nome': nome,
        'código': codigo,
        'preço': preco,
        'descrição': descricao,
        'disponibilidade': disponibilidade,
        'disponibilidade_entrega': disponibilidade_entrega,
        'estoque': estoque,
        'imagens': imagens
    }

In [23]:
# obter soup de cada url de cada produto


df_produtos_consolidado = pd.DataFrame()
counter = 0
for link in tqdm(urls["links_catalogo"], desc='Obtendo dados dos produtos', unit='produto'):

    ## baixar pagina do produto
    prod_content= requests.get(link).text
    ### converter página em soup
    prod_content_soup = bs(prod_content, 'html.parser')
    ### extrair dicionário com os dados chave do produto ## converter dicionário em dataframe
    dict_produto = extract_data_from_html(prod_content_soup)
    ## converter a lita de imagens em str
    dict_produto["imagens"] = str(dict_produto["imagens"])

    ##criar um dataframe do produto
    df_produto = pd.DataFrame(dict_produto, index=[0])
    ###inserir o link do produto
    df_produto["link"] = link
    ### inseirr a data de extração
    df_produto["data_extracao"] = datetime.now()
    ## adicionar ao dataframe consolidado
    df_produtos_consolidado = pd.concat([df_produtos_consolidado, df_produto], ignore_index=True)
    ##colocar uma pausa de 1 segundo a cada 50 produtos
    counter += 1
    time.sleep(5)
    if counter  == 201:
        time.sleep(57)
        counter = 0
        

# salvar dataframe consolidado em csv

df_produtos_consolidado.to_csv('data/produtos_consolidado.csv', index=False, sep='|')

### salvar dataframe consolidado no banco de dados



Obtendo dados dos produtos: 100%|██████████| 1511/1511 [2:57:25<00:00,  7.05s/produto]  


In [21]:
index

NameError: name 'index' is not defined

In [2]:

p_host = 'localhost'
p_port = 5432
db = 'postgres'
ssh = True
ssh_user = 'ubuntu'
ssh_host = '144.22.150.9'
psql_user = 'postgres'


psql_pass = 'alice11'
ssh_pkey = r"C:\Users\bcesa\OneDrive\Documentos\Infra na Núvem\mydata\ssh-key-2022-10-28.key"

pgres = dw.Postgresql_connect(pgres_host=p_host, pgres_port=p_port, db=db, ssh=ssh, ssh_user=ssh_user, ssh_host=ssh_host, ssh_pkey=ssh_pkey, psql_user=psql_user
                              , psql_pass=psql_pass)
#initiates a connection to the PostgreSQL database. In this instance we use ssh and must specify our ssh credentials.

#You'll need to define psql_user and psql_pass using input() and getpass() to temporarily store your credentials.
#Alternatively, best practice you may be to store your credentials as environment variables.
# psql_user = input("Please enter your database username:")


Server connected via SSH || Local Port: 63752...


In [4]:
df_produtos_consolidado = pd.read_csv('data/produtos_consolidado.csv', sep='|')

In [8]:
### criar conexão com o banco de dados

pgres.replace_table(df_produtos_consolidado, 'catalogo_shopmix')

Database [postgres] session created...
<> Table Replaced <>


2023-07-14 08:25:23,878| ERROR   | Socket exception: Foi forçado o cancelamento de uma conexão existente pelo host remoto (10054)


In [7]:
! pip install psycopg2

     ---------------------------------------- 1.2/1.2 MB 4.3 MB/s eta 0:00:00


In [ ]:


sql_statement = """
SELECT *
FROM catalogo_shopmix
;
"""
query_df = pgres.query( query=sql_statement)
query_df
#returns the results of an sql statement as a pandas dataframe. 
#This example returns the column names and data types of table 'ey_test_table'.

Database [postgres] session created...
<> Query Sucessful <>


,link


2023-07-10 14:13:30,268| ERROR   | Socket exception: Foi forçado o cancelamento de uma conexão existente pelo host remoto (10054)
